# LSTM Based Language Model - Part II
A language model looks at the context to generate next set of words. This context is also called as a sliding window which moves across the input sentence from left to right(right to left for language which are written from right to left). 

This is the second notebook with same layout. We present two variants of the model, one with stacked LSTM layers and one with a bidirectional LSTM layer.


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PacktPublishing/Hands-On-Generative-AI-with-Python-and-TensorFlow-2/blob/master/Chapter_9/language_model_stacked_lstm.ipynb)

## Import Required Libraries

In [34]:
import os
import math
import numpy as np
import tensorflow as tf

In [35]:
print("Tensorflow version={}".format(tf.__version__))

Tensorflow version=2.10.1


## Load Dataset

In [36]:
# https://www.gutenberg.org/ebooks/2600
datafile_path = r'gabungan.txt'

In [37]:
# Load the text file
text = open(datafile_path, 'rb').read().decode(encoding='utf-8')
print ('Book contains a total of {} characters'.format(len(text)))

Book contains a total of 77309 characters


In [38]:
idx = 8091
print(text[idx:idx+500])

ah-mudahan ada mawar yang bisa kupetik dan kupersembahkan kepadamu.
Sayangku, tunggulah aku.

Sejak aku melihatmu tuk pertama kali, aku merasakan getaran yang berbeda. Betapa aku tak menyadarinya. Namun, setelah kesekian kali aku melihatmu, aku sadar, selama ini aku menyimpan rasa sama kamu. Rasanya seperti ada yang hilang jika sehari saja ga melihat kamu.

Kamu harus tahu, betapa senangnya aku, jika tatap mata ini berbalas. Bukan hanya itu, aku juga suka tawamu, candamu dan senyumanmu itu.


In [39]:
# We remove first 8k characters to remove 
# details related to project gutenberg
text = text [8091:]

In [40]:
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

63 unique characters


## Prepare Dataset
+ Dictionary of character to index mapping
+ Inverse mapping of index to character mapping

In [41]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [42]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  "'" :   3,
  ',' :   4,
  '-' :   5,
  '.' :   6,
  '1' :   7,
  '2' :   8,
  '3' :   9,
  '5' :  10,
  '6' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'H' :  18,
  'I' :  19,
  ...
}


### Sample Output

In [43]:
print ('{} ---- char-2-int ----  {}'.format(repr(text[40:60]), text_as_int[40:60]))

'an kupersembahkan ke' ---- char-2-int ----  [33 46  2 43 52 48 37 49 50 37 45 34 33 40 43 33 46  2 43 37]


### Prepare Batch of Training Samples
+ Sequence length limit to 100
+ Use ``tf.data`` API to prepare batches

In [44]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
    print(idx2char[i.numpy()])

a
h
-
m
u
d
a
h
a
n


In [45]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(10):
    print(repr(''.join(idx2char[item.numpy()])))
    print("-"*110)

'ah-mudahan ada mawar yang bisa kupetik dan kupersembahkan kepadamu.\r\nSayangku, tunggulah aku.\r\n\r\nSeja'
--------------------------------------------------------------------------------------------------------------
'k aku melihatmu tuk pertama kali, aku merasakan getaran yang berbeda. Betapa aku tak menyadarinya. Na'
--------------------------------------------------------------------------------------------------------------
'mun, setelah kesekian kali aku melihatmu, aku sadar, selama ini aku menyimpan rasa sama kamu. Rasanya'
--------------------------------------------------------------------------------------------------------------
' seperti ada yang hilang jika sehari saja ga melihat kamu.\r\n\r\nKamu harus tahu, betapa senangnya aku, '
--------------------------------------------------------------------------------------------------------------
'jika tatap mata ini berbalas. Bukan hanya itu, aku juga suka tawamu, candamu dan senyumanmu itu. Kala'
--------------------------

### Prepare Input->Target samples

In [46]:
def split_input_target(chunk):
    """
    Utility which takes a chunk of input text and target as one position shifted form of input chunk.
    Parameters:
        chunk: input list of words
    Returns:
        Tuple-> input_text(i.e. chunk minus last word),target_text(input chunk minus the first word)
    """
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [47]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'ah-mudahan ada mawar yang bisa kupetik dan kupersembahkan kepadamu.\r\nSayangku, tunggulah aku.\r\n\r\nSej'
Target data: 'h-mudahan ada mawar yang bisa kupetik dan kupersembahkan kepadamu.\r\nSayangku, tunggulah aku.\r\n\r\nSeja'


In [48]:
# Batch size
BATCH_SIZE = 128
# Buffer size to shuffle the dataset
BUFFER_SIZE = 10000

In [49]:
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print("Dataset Shape={}".format(dataset))

Dataset Shape=<BatchDataset element_spec=(TensorSpec(shape=(128, 100), dtype=tf.int32, name=None), TensorSpec(shape=(128, 100), dtype=tf.int32, name=None))>


## Prepare Language Model

In [50]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size,is_bidirectional=False):
    """
    Utility to create a model object.
    Parameters:
        vocab_size: number of unique characters
        embedding_dim: size of embedding vector. This typically in powers of 2, i.e. 64, 128, 256 and so on
        rnn_units: number of LSTM units to be used
        batch_size: batch size for training the model
    Returns:
        tf.keras model object
    """
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]))
    if is_bidirectional:
      model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform')))
    else:
      model.add(tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'))
    model.add(tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'))
    model.add(tf.keras.layers.Dense(vocab_size))
    return model

### Define the Model Parameters

In [51]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [52]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [53]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (128, None, 256)          16128     
                                                                 
 lstm_4 (LSTM)               (128, None, 1024)         5246976   
                                                                 
 lstm_5 (LSTM)               (128, None, 1024)         8392704   
                                                                 
 dense_2 (Dense)             (128, None, 63)           64575     
                                                                 
Total params: 13,720,383
Trainable params: 13,720,383
Non-trainable params: 0
_________________________________________________________________


In [54]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [55]:
model.compile(optimizer='adam', loss=loss)

### Setup Callbacks

In [56]:
# Directory where the checkpoints will be saved
checkpoint_dir = r'data/training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [57]:
EPOCHS = 100
#history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

## Generate Fake Text

### Load Latest Checkpoint

In [58]:
# fetch the latest checkpoint from the model directory
tf.train.latest_checkpoint(checkpoint_dir)

'data/training_checkpoints\\ckpt_100'

In [59]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [60]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (1, None, 256)            16128     
                                                                 
 lstm_6 (LSTM)               (1, None, 1024)           5246976   
                                                                 
 lstm_7 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 dense_3 (Dense)             (1, None, 63)             64575     
                                                                 
Total params: 13,720,383
Trainable params: 13,720,383
Non-trainable params: 0
_________________________________________________________________


### Utility Function to Generate Text

In [61]:
def generate_text(model, mode='greedy', context_string='Hello', num_generate=1000, 
                  temperature=1.0):
    """
    Utility to generate text given a trained model and context
    Parameters:
        model: tf.keras object trained on a sufficiently sized corpus
        mode: decoding mode. Default is greedy. Other mode is
              sampling (set temperature)
        context_string: input string which acts as context for the model
        num_generate: number of characters to be generated
        temperature: parameter to control randomness of outputs
    Returns:
        string : context_string+text_generated
    """

    # vectorizing: convert context string into string indices
    input_eval = [char2idx[s] for s in context_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # String for generated characters
    text_generated = []
    beam_input_predictions = []
    model.reset_states()
    # Loop till required number of characters are generated
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        if mode == 'greedy':
          predicted_id = np.argmax(predictions[0])
          
        elif mode == 'sampling':
          # temperature helps control the character returned by the model.
          predictions = predictions / temperature
          # Sampling over a categorical distribution
          predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # predicted character acts as input for next step
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])
    return (context_string + ''.join(text_generated))

### Greedy Decoding

In [62]:
print(generate_text(model, mode= 'greedy', context_string="Assalamu'alaikum",num_generate=1000))

Assalamu'alaikummu, setira kita bersama. Tapi hati ini merasa sendiri ketika kau tak di sisiku…aku begitu merindukanmu.

Kusadari, mungkin ini yang disebut cinta. Yah . . .ku beranikan diri ku katakan bahwa aku mencintaimu, sangat mencintaimu.

Terserah apa yang ada di benakmu saat ini tentangku. Namun, aku berharap balasan surat darimu yang akan menyejukkan hatiku.

Yang mencintaimu,

Aku tidak mengerti apa itu cinta dan bagaimana rasa sayang bisa terjadi.

Namun, kau hadir mengubah semuanya menjadi lebih indah. Tatkala hati ini beku, kaulah yang telah mencairkannya dengan penuh kelembutan layaknya anak panah yang menembus palung terdalam.

Kadang-kadang, terbersit dalam pikiran bahwa cinta itu aneh. Ada yang mengatakan kalau cinta itu tidak memiliki kaki, tapi cinta dapat berjalan dari hati ke hati, dan kini cintaku tlah berjalan ke hatimu. Kini cinta ini tlah lumpuh untuk tetap berada di hatimu, tidak akan pernah sanggup untuk berjalan meninggalkanmu.

Kasih. . .cintaku itu sederhan

### Sampled Decoding

In [63]:
print(generate_text(model, mode= 'sampling', context_string=u"sayang ku",num_generate=1000,temperature=0.2))

sayang ku…

Sayangku, lebih baik bagiku berjalan bersamamu menyusuri pematang di pesawahan di lereng bukit di sebelah Barat desamu daripada memandangi kota Paris dari atas Menara Eiffel. Karena di pesawahan itu, aku dapat menggennggam jemarimu dan membaui aroma lembut rambutmu. Sedangkan di Eiffel, aku hanya dapat membayangkan wajahmu yang selalu menyejukkan hatiku.

Sayangku, terasa lebih nikmat bagiku, saat menyantap maso malang semangkuk berdua dengan tempias hujan sekali-kali mengelus muka kita, daripada makan di restoran mewah di Paris bersama bos. Di baso malang aku puas mengumpulkan senyummu sementara di Paris aku selalu teringat padamu.

Sayangku, waktu serasa berhenti apabila kuhitung hari ku kan kembali. Lama sekali rasanya. Kupikir bukan lagi sehari rasa seminggu tetapi sehari rasanya berabad-abad yang harus kulewati.

Sayangku, kuharap Engkau sudi sabar menunggu. Tiga bulan lagi aku akan selesai dengan tugasku di sini. Saat hari tiu tiba nanti, aku akan langsung menemuimu. 

In [64]:
print(generate_text(model, mode= 'sampling', context_string=u"rindu",num_generate=100,temperature=0.6))

rindu.

Sejak pertama kali aku mengenalmu, tak ada yang berbeda dari rasa ini. Tak ada yang berbeda dar


In [65]:
print(generate_text(model, mode= 'sampling', context_string=u"malam",num_generate=100,temperature=0.9))

malam surat ini sebagai bentuk rasa rinduku padamu. Surat ini aku buat ketika cuaca dan pagakan kita basa
